In [10]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import yaml
from easydict import EasyDict as edict
import torch
from torch import nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pyspng

# Functions
def yaml_load(fileName):
    dict_config = None
    with open(fileName, 'r') as ymlfile:
        dict_config = edict(yaml.safe_load(ymlfile))

    return dict_config


class Dataset_instance(Dataset):

    def __init__(self, wsi_path_patches, transform=None, preprocess=None):

        self.wsi_path_patches = wsi_path_patches
        self.transform = transform
        self.preprocess = preprocess


    def __len__(self):
        return len(self.wsi_path_patches)

    def __getitem__(self, index):
        
        # Load the patch image saved as png (key)
        with open(self.wsi_path_patches[index][0], 'rb') as fin:
            key =  pyspng.load(fin.read())

        if self.transform:
            query = self.transform(image=key)['image']
        else:
            query = key

        if self.preprocess:
            query = self.preprocess(query).type(torch.FloatTensor)
            key = self.preprocess(key).type(torch.FloatTensor)
        
        return key, query


class ModelOption():
    def __init__(self, model_name: str,
                 num_classes: int,
                 freeze=False,
                 num_freezed_layers=0,
                 dropout=0.0, 
                 embedding_bool=False):

        self.model_name = model_name
        self.num_classes = num_classes
        self.freeze = freeze
        self.num_freezed_layers = num_freezed_layers
        self.dropout = dropout
        self.embedding_bool = embedding_bool

        if self.model_name.lower() == "resnet50":
            """ ResNet50 """
            self.net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

            if self.freeze:
                # Freezing the number of layers
                # ResNet has two named layers in position 2 and 3 named relu and maxpool
                # that do not have any learnable parameters, therefore if the user wants to
                # freeze more than 2 layers we offset the num_freezed_layers with two
                if self.num_freezed_layers > 2:
                    self.net = self.set_parameter_requires_grad(self.net,
                                                                self.num_freezed_layers + 2)
                else:
                    self.net = self.set_parameter_requires_grad(self.net,
                                                                
                                                                self.num_freezed_layers)

            self.input_features = self.net.fc.in_features  # 2048
            
            # self.net.fc = nn.Sequential(nn.Dropout(p=self.dropout),
            #                             nn.Linear(input_features,
            #                                       input_features // 4),
            #                             nn.ReLU(inplace=True),
            #                             nn.Dropout(p=self.dropout),
            #                             nn.Linear(input_features // 4,
            #                                      input_features // 8),
            #                             nn.ReLU(inplace=True),
            #                             nn.Dropout(p=self.dropout),
            #                             nn.Linear(input_features // 8,
            #                                      self.num_classes))

            self.resize_param = 224

        elif self.model_name.lower() == "convnext":
            """ ConvNeXt small """
            self.net = models.convnext_small(weights='DEFAULT')

            if self.freeze:
                # Freezing the number of layers
                self.net.features = self.set_parameter_requires_grad(self.net.features,
                                                                     self.num_freezed_layers)

            input_features = self.net.classifier[2].in_features  # 768
            self.net.classifier[2] = nn.Sequential(nn.Dropout(p=self.dropout),
                                                   nn.Linear(input_features,
                                                             input_features // 2),
                                                   nn.ReLU(inplace=True),
                                                   nn.Dropout(p=self.dropout),
                                                   nn.Linear(input_features // 2,
                                                             input_features // 4),
                                                   nn.ReLU(inplace=True),
                                                   nn.Dropout(p=self.dropout),
                                                   nn.Linear(input_features // 4,
                                                             self.num_classes))

            self.resize_param = 224

        elif self.model_name.lower() == "swin":
            """ Swin Transformer V2 -T """
            self.net = models.swin_v2_t(weights=models.Swin_V2_T_Weights.DEFAULT)

            if self.freeze:
                # Freezing the number of layers
                self.net.features = self.set_parameter_requires_grad(self.net.features,
                                                                     self.num_freezed_layers)

            input_features = self.net.head.in_features  # 768
            self.net.head = nn.Sequential(nn.Dropout(p=self.dropout),
                                          nn.Linear(input_features,
                                                    input_features // 2),
                                          nn.ReLU(inplace=True),
                                          nn.Dropout(p=self.dropout),
                                          nn.Linear(input_features // 2,
                                                    input_features // 4),
                                          nn.ReLU(inplace=True),
                                          nn.Dropout(p=self.dropout),
                                          nn.Linear(input_features // 4,
                                                    self.num_classes))

            self.resize_param = 224

        elif self.model_name.lower() == "efficient":
            """ EfficientNet b0 """
            self.net = models.efficientnet_b0(weights='DEFAULT')

            if self.freeze:
                # Freezing the number of layers
                self.net.features = self.set_parameter_requires_grad(self.net.features,
                                                                     self.num_freezed_layers,
                                                                     feature_layers=9)

            input_features = self.net.classifier[1].in_features  # 1200
            self.net.classifier = nn.Sequential(nn.Dropout(p=self.dropout),
                                                nn.Linear(input_features,
                                                          input_features // 2),
                                                nn.ReLU(inplace=True),
                                                nn.Dropout(p=self.dropout),
                                                nn.Linear(input_features // 2,
                                                          input_features // 4),
                                                nn.ReLU(inplace=True),
                                                nn.Dropout(p=self.dropout),
                                                nn.Linear(input_features // 4,
                                                          self.num_classes))

            self.resize_param = 224

        else:
            print("Invalid model name, MODEL NOT LOAD")
            TypeError("Valid model names are 'resnet', 'convnext', 'swim' or 'efficient'")
            exit()

    def set_parameter_requires_grad(model, number_frozen_layers, feature_layers=8):
        for k, child in enumerate(model.named_children()):
            if k == number_frozen_layers or k == feature_layers:
                break
            for param in child[1].parameters():
                param.requires_grad = False

        return model
    

class Encoder(torch.nn.Module):
    def __init__(self, model, dim):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(Encoder, self).__init__()

        self.model = model
        self.fc_input_features = self.model.input_features
        self.num_classes = self.model.num_classes
        self.dim = dim
        self.net = self.model.net

        self.conv_layers = torch.nn.Sequential(*list(self.net.children())[:-1])

        if (torch.cuda.device_count()>1):
            self.conv_layers = torch.nn.DataParallel(self.conv_layers)

        if self.model.embedding_bool:

            if ('resnet34' in self.model.model_name):
                self.E = self.dim
                self.L = self.E
                self.D = 64
                self.K = self.num_classes

            elif ('resnet50' in self.model.model_name):
                self.E = self.dim
                self.L = self.E
                self.D = 128
                self.K = self.num_classes

            elif ('resnet152' in self.model.model_name):
                self.E = self.dim
                self.L = self.E
                self.D = 128
                self.K = self.num_classes

            self.embedding = torch.nn.Linear(in_features=self.fc_input_features, out_features=self.E)
            self.post_embedding = torch.nn.Linear(in_features=self.E, out_features=self.E)

        self.prelu = torch.nn.PReLU(num_parameters=1, init=0.25) 
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        #if used attention pooling
        A = None
        #m = torch.nn.Softmax(dim=1)
        dropout = torch.nn.Dropout(p=0.2)
        relu = torch.nn.ReLU()
        tanh = torch.nn.Tanh()

        if x is not None:
            #print(x.shape)
            conv_layers_out = self.conv_layers(x)
            #print(x.shape)

            conv_layers_out = conv_layers_out.view(-1, self.fc_input_features)

        if self.model.embedding_bool:
            #embedding_layer = self.relu(conv_layers_out)
            embedding_layer = self.embedding(conv_layers_out)
            embedding_layer = self.relu(embedding_layer)
            embedding_layer = self.post_embedding(embedding_layer)

            features_to_return = embedding_layer

        else:
            features_to_return = conv_layers_out

        norm = torch.norm(features_to_return, p='fro', dim=1, keepdim=True)

        #normalized_array = features_to_return #/ norm
        #normalized_array = torch.nn.functional.normalize(features_to_return, dim=1)
        #normalized_array = torch.norm(features_to_return, p='fro', dim=1, keepdim=True)
        normalized_array = features_to_return 

        return normalized_array




In [12]:
thispath = Path.cwd().resolve()

datadir = Path(thispath.parent / "data")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load PyTorch model
experiment_name = "MoCo_try_Adam"

modeldir = Path(thispath.parent / "trained_models" / experiment_name)

cfg = yaml_load(modeldir / f"config_{experiment_name}.yml")

model = ModelOption(cfg.model.model_name,
                cfg.model.num_classes,
                freeze=cfg.model.freeze_weights,
                num_freezed_layers=cfg.model.num_frozen_layers,
                dropout=cfg.model.dropout,
                embedding_bool=cfg.model.embedding_bool
                )    

# Encoder and momentum encoder
moco_dim = cfg.training.moco_dim

encoder = Encoder(model, dim=moco_dim).to(device)

optimizer = getattr(torch.optim, cfg.training.optimizer)
optimizer = optimizer(encoder.parameters(), **cfg.training.optimizer_args)

# checkpoint = torch.load(modeldir / cfg.dataset.magnification / cfg.model.model_name / f"{experiment_name}.pt")
checkpoint = torch.load(modeldir / cfg.dataset.magnification / cfg.model.model_name / "MoCo.pt")
encoder.load_state_dict(checkpoint["encoder_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
loss = checkpoint["loss"]
epoch = checkpoint["epoch"] + 1

print(f"Loaded encoder using as backbone {cfg.model.model_name} with a best loss of {loss} at Epoch {epoch}")
print(optimizer.param_groups[0]['lr'])

# Load patches
pyhistdir = Path(datadir / "Mask_PyHIST_v2")

selected_wsi = ["000030303300314205", "000030494900323685"]

dataset_path = natsorted([i for i in pyhistdir.rglob("*_densely_filtered_paths.csv") if selected_wsi[0] in str(i) or selected_wsi[1] in str(i)])

number_patches = 0
path_patches = []
for wsi_patches in tqdm(dataset_path, desc="Selecting patches to check model"):

    csv_instances = pd.read_csv(wsi_patches).to_numpy()
    
    path_patches.extend(csv_instances)

selected_patches = ["000030303300314205_06422", "000030303300314205_06423", "000030303300314205_06444" ,"000030494900323685_02127", "000030494900323685_02770", "000030494900323685_02513"]
selected_patches_path = []
for patch in path_patches:
    if any(item in str(patch) for item in selected_patches):
        selected_patches_path.append(patch)

preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=cfg.dataset.mean, std=cfg.dataset.stddev),
        transforms.Resize(size=(model.resize_param, model.resize_param))
    ])

params_instance = {'batch_size': 1,
                           'shuffle': False,
                           'pin_memory': True,
                           'num_workers': 2}

instances = Dataset_instance(selected_patches_path, transform=None, preprocess=None)
generator = DataLoader(instances, **params_instance)

encoder.eval()
# do not accumulate gradients (faster)
with torch.no_grad():
    
    for x_q, x_k in generator:

        img_q = x_q.squeeze().numpy()
        plt.imshow(img_q)
        plt.title("Patch")
        plt.show()

        img_k = x_k.squeeze().numpy()
        plt.imshow(img_k)
        plt.show()

        # x_q, x_k = x_q.to(device, non_blocking=True), x_k.to(device, non_blocking=True)

        # q = encoder(x_q) # q : (N, 128)

/home/lluis/miniconda3/envs/hlung/lib/python3.9/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


Loaded encoder using as backbone resnet50 with a best loss of 7.036789032876164 at Epoch 19
0.01


Selecting patches to check model: 100%|██████████| 2/2 [00:00<00:00, 190.17it/s]
